In [ ]:
from datetime import datetime
from ripe.atlas.cousteau import AtlasRequest, AtlasResultsRequest
from ripe.atlas.sagan import Result, DnsResult

from ipwhois import IPWhois
STOPPED = 4


def get_measurements_collection_by_date_and_type(date, measurement_type):
    timestamp = datetime.strptime(date, "%Y-%m-%d").timestamp()
    filters = {"type": measurement_type,
               "status": STOPPED,
               "start_time__gte": timestamp,
               "start_time__lt": timestamp+24*60*60,
               "fields": "id,start_time,stop_time,af,query_argument,query_type,result"
               }
    url_path = "/api/v2/measurements/"
    is_success, response = AtlasRequest(
        **{"url_path": url_path}).get(**filters)
    if is_success:
        return response["results"]
    else:
        return None


def get_measurement_results(measurement_id):
    url_path = f"/api/v2/measurements/{measurement_id}/results/"
    is_success, response_results = AtlasRequest(
        **{"url_path": url_path}).get()
    if is_success:
        return response_results
    else:
        return None


def get_probes_geolocation_by_id_list(probe_id_list):
    url_path = "/api/v2/probes/"
    result = []
    for sub_list in [probe_id_list[i:i+500] for i in range(0, len(probe_id_list), 500)]:
        is_success, response_results = AtlasRequest(
            **{"url_path": url_path}).get(**{"id__in": ",".join(sub_list)})
        if is_success:
            result.extend(                  response_results["results"])
        else:
            continue
    return result


def map_measurement_result_to_probe_info(parsed_result):
    return {
        "probe_id": parsed_result.probe_id,
        "rtt": parsed_result.responses[0].response_time,
    }


def create_measurement_hash(measurement_id):
    
    def find_object_by_id(collection, id):
        for obj in collection:
            if obj["id"] == id:
                return obj  # Return the object if found
            
    meausrement_results = get_measurement_results(measurement_id)
    probes_information = [
        map_measurement_result_to_probe_info(DnsResult(result))
        for result in meausrement_results if not DnsResult(result).is_error]
    probes_geolocation = get_probes_geolocation_by_id_list(
        [str(probe_obj["probe_id"]) for probe_obj in probes_information])
    for probe_info in probes_information:
        according_probe_geolocation = find_object_by_id(
            probes_geolocation, probe_info["probe_id"])
        probe_info


def check_dns_measurements(date):
    measurements = get_measurements_collection_by_date_and_type(date, "dns")
    for measurement in measurements:
        print('Measurement ID:', measurement.get("id"))
        print('Start Time:', datetime.fromtimestamp(
            measurement.get("start_time")))
        print('Stop Time:', datetime.fromtimestamp(
            measurement.get("stop_time")))
        print(
            f"Query: {measurement.get('af')} {measurement.get('query_argument')} {measurement.get('query_type')}")
        print('Results Link:', measurement.get("result"))
        print('----')
        measurement_hash = create_measurement_hash(measurement.get("id"))
        print()
        # meausrement_results = get_measurement_results(measurement.get('id'))
        # probes_information = [
        #     map_measurement_result_to_probe_info(DnsResult(result))
        #     for result in meausrement_results if not DnsResult(result).is_error]

        # probes_geolocation=get_probes_geolocation_by_id_list([str(probe_obj["probe_id"]) for probe_obj in probes_information])
        # for res in meausrement_results:
        #     parsed_result = DnsResult(res)
        #     if not parsed_result.is_error:
        #         is_success, results = AtlasRequest(
        #             **{"url_path": f"/api/v2/probes/{parsed_result.probe_id}/"}).get()
        #         print('DNS Response Time:',
        #               parsed_result.responses[0].response_time)
        #         print('Origin IP:', parsed_result.origin)
        #         # print('DNS Response Time:', parsed_result.response_time)
        #         print('DNS Result:', parsed_result.result)
        #         print('----')
        # else:
        #     print('Failed to get the results for the measurement with ID:', result.get("id"))


check_dns_measurements("2022-05-01")

In [73]:
from datetime import datetime, timedelta
from ripe.atlas.cousteau import AtlasRequest
import pycountry
import sys
sys.path.append("../")
from Backend.RAICAT.probes_db import (probes_data)

def get_all_countries():
    country_list = [country.alpha_2 for country in pycountry.countries]
    return country_list

def probes_ipv6_check(probe, start_date, finish_date):
    filters = {
        "probe": probe,
        "date__gte": start_date,
        "date__lte": finish_date,
    }
    url_path = "/api/v2/probes/archive"
    is_success, response = AtlasRequest(**{"url_path": url_path}).get(
        **filters
    )
    if is_success:
        return [
            {
                "id": result["id"],
                "asn_v6": result["asn_v6"],
                "asn_v4": result["asn_v4"],
                "date": result["date"],
            }
            for result in response["results"]
        ]
    else:
        return None
    
def get_probes_for_country(country_code):
    filters = {"country_code": country_code}
    url_path = "/api/v2/probes/"
    is_success, response = AtlasRequest(**{"url_path": url_path}).get(**filters)
    if is_success:
        return response["results"]
    else:
        return None
    


def add_results_ipv6(data, day, percentage):
    if percentage!=0:
        data.append({"name": day, "ipv6": percentage})
    return data

def check_as_for_probes(country_code, start_date, finish_date):
    #probes = get_probes_for_country(country_code)
    start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    end_datetime = datetime.strptime(finish_date, "%Y-%m-%d")
    delta = end_datetime - start_datetime
    #ids = [item["id"] for item in probes]
    ids = [item["id"] for item in probes_data if item["country_code"] == "IL"]
    results = probes_ipv6_check(ids, start_date, finish_date)
    data = []
    for i in range(delta.days + 1):
        current_date = start_datetime + timedelta(days=i)
        # date_obj = datetime.strptime(current_date, "%Y%m%d")
        current_day = current_date.strftime("%Y%m%d")
        as_version_6 = set()
        as_version_4 = set()
        for res in results:
            if res["date"] == current_day:
                if res["asn_v6"] == None:
                    as_version_4.add(res["asn_v4"])
                else:
                    as_version_6.add(res["asn_v6"])
        amount_as_ipv6 = len(as_version_6)
        amount_as_ipv4 = len(as_version_4)
        # print(amount_as_ipv4, "ipv4")
        # print(amount_as_ipv6, "ipv6")
        percentage = (
            (amount_as_ipv6 / (amount_as_ipv6 + amount_as_ipv4)) * 100
            if amount_as_ipv6 + amount_as_ipv4 != 0
            else 0
        )
        # ! TODO fix default value 
        # print(f"At date {current_day} the percentage of IPv6 ASes in {country_code} is: {percentage}%")
        current_day = current_date.strftime("%Y-%m-%d")
        data = add_results_ipv6(data, current_day, percentage)
    return data

def get_probes_check_one_day(probe, date):
    filters = {"probe": probe,
               "date": date,
               }
    url_path = "/api/v2/probes/archive"
    is_success, response = AtlasRequest(
        **{"url_path": url_path}).get(**filters)
    if is_success:
        return [{
                "id":result["id"],
                "asn_v6":result["asn_v6"],
                "asn_v4":result["asn_v4"],
                "date":result["date"],
                } 
                for result in response["results"]    
                ]
    else:
        return None

def check_delta_per_country(start_date, finish_date):
    country_list = get_all_countries()
    #print(country_list)
    #country_list = [
    #'AT', 'BE', 'BG', 'HR', 'CY', 'CZ', 'DK', 'EE', 'FI', 'FR', 'DE', 'GR',
    #'HU', 'IE', 'IT', 'LV', 'LT', 'LU', 'MT', 'NL', 'PL', 'PT', 'RO', 'SK',
    #'SI', 'ES', 'SE']
    data = []
    for country in country_list:
        probes = get_probes_for_country(country)
        #ids = [item["id"] for item in probes]
        ids = [item["id"] for item in probes_data]
        start_result = get_probes_check_one_day(ids, start_date)
        finish_result = get_probes_check_one_day(ids, finish_date)
        as_version_6_start = set()
        as_version_4_start = set()
        as_version_6_finish = set()
        as_version_4_finish = set()
        if start_result != None: 
            for res in start_result: 
                if (res["asn_v6"]==None):
                    as_version_4_start.add(res["asn_v4"])
                else:
                    as_version_6_start.add(res["asn_v6"])
        if finish_result != None:
            for res in finish_result: 
                if (res["asn_v6"]==None):
                    as_version_4_finish.add(res["asn_v4"])
                else:
                    as_version_6_finish.add(res["asn_v6"])
        amount_as_ipv6_start = len(as_version_6_start)
        amount_as_ipv4_start = len(as_version_4_start)
        #percentage_start = (amount_as_ipv6_start / (amount_as_ipv6_start + amount_as_ipv4_start)) * 100
        percentage_start = (
            (amount_as_ipv6_start / (amount_as_ipv6_start + amount_as_ipv4_start)) * 100
            if amount_as_ipv6_start + amount_as_ipv4_start != 0
            else 0
        )
        amount_as_ipv6_finish = len(as_version_6_finish)
        amount_as_ipv4_finish = len(as_version_4_finish)
        #percentage_finish = (amount_as_ipv6_finish / (amount_as_ipv6_finish + amount_as_ipv4_finish)) * 100
        percentage_finish = (
            (amount_as_ipv6_finish / (amount_as_ipv6_finish + amount_as_ipv4_finish)) * 100
            if amount_as_ipv6_finish + amount_as_ipv4_finish != 0
            else 0
        )
        delta = percentage_finish - percentage_start
        data.append((country, percentage_start, percentage_finish, delta))
    return data




#print(get_probes_for_country("IL"))
#print(probes_ipv6_check(35104, datetime.strptime("2023-07-07", "%Y-%m-%d"), datetime.strptime("2023-07-08", "%Y-%m-%d")))
#probes = get_probes_for_country("IL")
#print(probes_ipv6_check(ids, "2023-07-07", "2023-07-08"))
#data = check_delta_per_country("2023-07-07", "2020-01-01")
#print(data)
print(check_as_for_probes("US", "2022-12-12", "2023-01-01"))
#ids = [item["id"] for item in probes_data if item["country_code"] == "IL"]

    

[{'name': '2022-12-12', 'ipv6': 30.76923076923077}, {'name': '2022-12-13', 'ipv6': 30.76923076923077}, {'name': '2022-12-14', 'ipv6': 30.76923076923077}, {'name': '2022-12-15', 'ipv6': 30.76923076923077}, {'name': '2022-12-16', 'ipv6': 30.76923076923077}, {'name': '2022-12-17', 'ipv6': 30.76923076923077}, {'name': '2022-12-18', 'ipv6': 30.76923076923077}, {'name': '2022-12-19', 'ipv6': 30.76923076923077}, {'name': '2022-12-20', 'ipv6': 30.76923076923077}, {'name': '2022-12-21', 'ipv6': 30.76923076923077}, {'name': '2022-12-22', 'ipv6': 30.76923076923077}, {'name': '2022-12-23', 'ipv6': 30.76923076923077}, {'name': '2022-12-24', 'ipv6': 30.76923076923077}, {'name': '2022-12-25', 'ipv6': 30.76923076923077}, {'name': '2022-12-26', 'ipv6': 30.76923076923077}, {'name': '2022-12-27', 'ipv6': 30.76923076923077}, {'name': '2022-12-28', 'ipv6': 30.76923076923077}, {'name': '2022-12-29', 'ipv6': 30.76923076923077}, {'name': '2022-12-30', 'ipv6': 30.76923076923077}, {'name': '2022-12-31', 'ipv6':

In [67]:
import pprint
new_data=[(item[0],item[1],item[2],item[1]-item[2])for item in data]
sorted_data = sorted(new_data,key=lambda x:x[3],reverse=True)
# pprint.pprint(sorted_data)
# %store sorted_data >ipv6_deltas.py

Writing 'sorted_data' (list) to file 'ipv6_deltas.py'.


In [ ]:

from pprint import pprint
from ipwhois.experimental import bulk_lookup_rdap,get_bulk_asn_whois
ip_list = ['74.125.225.229', '2001:4860:4860::8888', '62.239.237.1', '2a00:2381:ffff::1', '210.107.73.73',
        #    '2001:240:10c:1::ca20:9d1d', '200.57.141.161', '2801:10:c000::', '196.11.240.215', '2001:43f8:7b0::', '133.1.2.5', '115.1.2.3'
           ]
# obj = IPWhois('["8.8.4.4", "1.1.1.1", "2c0f:fb50:4003::"]')
results= get_bulk_asn_whois(addresses=ip_list)
# results, stats = bulk_lookup_rdap(addresses=ip_list)
pprint(results)

In [6]:
import pycountry

def get_all_countries():
    country_list = [country.alpha_2 for country in pycountry.countries]
    return country_list

print(get_all_countries())

['AW', 'AF', 'AO', 'AI', 'AX', 'AL', 'AD', 'AE', 'AR', 'AM', 'AS', 'AQ', 'TF', 'AG', 'AU', 'AT', 'AZ', 'BI', 'BE', 'BJ', 'BQ', 'BF', 'BD', 'BG', 'BH', 'BS', 'BA', 'BL', 'BY', 'BZ', 'BM', 'BO', 'BR', 'BB', 'BN', 'BT', 'BV', 'BW', 'CF', 'CA', 'CC', 'CH', 'CL', 'CN', 'CI', 'CM', 'CD', 'CG', 'CK', 'CO', 'KM', 'CV', 'CR', 'CU', 'CW', 'CX', 'KY', 'CY', 'CZ', 'DE', 'DJ', 'DM', 'DK', 'DO', 'DZ', 'EC', 'EG', 'ER', 'EH', 'ES', 'EE', 'ET', 'FI', 'FJ', 'FK', 'FR', 'FO', 'FM', 'GA', 'GB', 'GE', 'GG', 'GH', 'GI', 'GN', 'GP', 'GM', 'GW', 'GQ', 'GR', 'GD', 'GL', 'GT', 'GF', 'GU', 'GY', 'HK', 'HM', 'HN', 'HR', 'HT', 'HU', 'ID', 'IM', 'IN', 'IO', 'IE', 'IR', 'IQ', 'IS', 'IL', 'IT', 'JM', 'JE', 'JO', 'JP', 'KZ', 'KE', 'KG', 'KH', 'KI', 'KN', 'KR', 'KW', 'LA', 'LB', 'LR', 'LY', 'LC', 'LI', 'LK', 'LS', 'LT', 'LU', 'LV', 'MO', 'MF', 'MA', 'MC', 'MD', 'MG', 'MV', 'MX', 'MH', 'MK', 'ML', 'MT', 'MM', 'ME', 'MN', 'MP', 'MZ', 'MR', 'MS', 'MQ', 'MU', 'MW', 'MY', 'YT', 'NA', 'NC', 'NE', 'NF', 'NG', 'NI', 'NU', 'NL

In [65]:
import file
from file import deltas_result as result

In [66]:
file.deltas_result

[('AW', 0.0, 0.0, 0.0),
 ('AF', 0.0, 0.0, 0.0),
 ('AO', 0.0, 0.0, 0.0),
 ('AI', 34.199525477347194, 31.560182544209926, 2.639342933137268),
 ('AX', 0.0, 0.0, 0.0),
 ('AL', 17.142857142857142, 18.51851851851852, -1.3756613756613767),
 ('AD', 0.0, 0.0, 0.0),
 ('AE', 44.44444444444444, 21.428571428571427, 23.015873015873016),
 ('AR', 30.508474576271187, 27.586206896551722, 2.9222676797194644),
 ('AM', 37.03703703703704, 40.0, -2.962962962962962),
 ('AS', 0.0, 0.0, 0.0),
 ('AQ', 0.0, 0.0, 0.0),
 ('TF', 34.199525477347194, 31.560182544209926, 2.639342933137268),
 ('AG', 0.0, 0.0, 0.0),
 ('AU', 57.49999999999999, 51.42857142857142, 6.071428571428569),
 ('AT', 62.0, 53.48837209302325, 8.511627906976749),
 ('AZ', 5.0, 5.88235294117647, -0.8823529411764701),
 ('BI', 12.5, 16.666666666666664, -4.166666666666664),
 ('BE', 50.0, 44.44444444444444, 5.555555555555557),
 ('BJ', 9.090909090909092, 10.0, -0.9090909090909083),
 ('BQ', 0.0, 0.0, 0.0),
 ('BF', 16.666666666666664, 0.0, 16.666666666666664),